# TOPSIS on pre-trained "Text Summarization" models
In this notebook we'll be using Text Summarization models from https://huggingface.co/models and apply them on 5 different text snippets in 3 different Domains (Politics, Medicine, Sports).

To check model performance we'll be using following metrics/parameters:
#### BLEU Score: 
BLEU (Bilingual Evaluation Understudy) is a metric for evaluating the quality of machine-generated text, commonly used in machine translation. It measures the precision of the generated text by comparing it to one or more reference texts.
#### BERTScore: 
BERT Score is a metric that assesses the quality of machine-generated text by comparing it to reference texts. It employs pre-trained BERT (Bidirectional Encoder Representations from Transformers) models to capture semantic similarity and evaluates the text on multiple levels of granularity.
#### METEOR Score:
METEOR (Metric for Evaluation of Translation with Explicit ORdering) is a metric for evaluating the quality of machine-generated text, particularly in machine translation. It considers precision, recall, and aligns the generated text with reference texts while also incorporating stemming and synonymy matching.
#### Jaccard Similarity: 
Jaccard Similarity measures the similarity between two sets by comparing the intersection and union of their elements. In natural language processing, it's often used to evaluate the similarity of two texts by considering the overlapping words.
#### Flesch Reading Ease: 
Flesch Reading Ease is a readability metric that assesses the complexity of written text. It provides a score based on the average sentence length and the average number of syllables per word, indicating how easy or difficult the text is to comprehend. Higher scores suggest easier readability.

Project Outline:
1. Import all necessary libraries and models from https://huggingface.co/models
2. Add text snippets in a list and generate summaries for each model
3. Apply metrics to check the performance of our models
4. Make dataframes for each model
5. group the dataframes according to Domain and Text
6. Apply topsis and determine which model works best for each domain

In [1]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from nltk.translate.bleu_score import corpus_bleu
from bert_score import score
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import jaccard_distance
from textstat import flesch_reading_ease
import warnings

warnings.filterwarnings("ignore")

/Users/chhavi/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


### Text Snipets for each Domain

In [3]:
politics_texts = ["In the ever-evolving landscape of global politics, diplomatic relations play a pivotal role in shaping the world order. Leaders from various nations navigate complex geopolitical challenges, seeking common ground to address shared concerns. The rise of multilateral platforms has transformed the nature of diplomacy, allowing for nuanced discussions on issues ranging from climate change to economic cooperation. As global citizens witness a dynamic interplay of power and influence, the importance of diplomatic efforts in fostering peace, stability, and international cooperation becomes increasingly evident. This text delves into the intricate web of global diplomacy, examining key diplomatic events and their impact on the interconnected world we inhabit.", 
                  "Elections stand as a cornerstone of democratic societies, providing citizens with the power to shape the course of their nations. In this exploration of the electoral process, we delve into the intricate dynamics of political campaigns, the role of media, and the impact of electoral systems on representation. As voters engage with candidates' platforms, ideologies clash, and diverse perspectives converge in the democratic arena. This text analyzes the complexities surrounding elections, highlighting their role in upholding democratic values and fostering civic participation.",
                  "The resurgence of populism has become a defining feature of contemporary political landscapes. This text examines the rise of populist movements across the globe, exploring their origins, ideologies, and implications for established political structures. From charismatic leaders to grassroots movements, populism manifests in various forms, challenging traditional notions of governance and political discourse. Analyzing the impact of populism on policy decisions, social cohesion, and international relations, this text seeks to unravel the complexities associated with this influential political phenomenon.",
                  "In the digital age, technology has reshaped the landscape of political campaigns, offering unprecedented tools for communication, mobilization, and voter engagement. This text delves into the innovative strategies employed by political campaigns, from social media outreach to data analytics. As candidates harness the power of technology to connect with diverse constituencies, questions arise about the ethical implications and potential consequences of this digital transformation. Navigating the intersection of technology and politics, this exploration sheds light on the evolving nature of political communication and campaigning.",
                  "The interconnectedness of nations in the 21st century necessitates a careful examination of international relations and the pursuit of global cooperation. From addressing transnational issues such as cybersecurity and pandemics to navigating geopolitical tensions, this text explores the multifaceted dimensions of international relations. Diplomacy, trade agreements, and international organizations all play crucial roles in shaping the responses to global challenges. As the world grapples with an array of complex issues, understanding the intricate dynamics of international relations becomes essential for fostering a more harmonious and collaborative global community."]
medicine_texts = ["Precision medicine represents a groundbreaking approach in healthcare, aiming to customize medical treatment based on individual patient characteristics. By leveraging genetic, genomic, and other patient-specific data, healthcare practitioners can tailor interventions to maximize efficacy while minimizing side effects. This text explores the exciting developments in precision medicine, highlighting how this innovative approach is revolutionizing the diagnosis and treatment of various medical conditions.", 
                  "Telemedicine, propelled by technological advancements, has emerged as a transformative force in healthcare delivery. This text delves into the expanding landscape of telehealth services, exploring how virtual consultations, remote monitoring, and digital health platforms are enhancing access to medical care. From urban centers to remote areas, telemedicine is bridging gaps in healthcare accessibility and empowering patients to actively participate in their well-being.", 
                  "Immunotherapy stands at the forefront of medical innovation, offering new avenues for treating various diseases, including cancer. This text examines the principles behind immunotherapy and explores recent breakthroughs in the field. From checkpoint inhibitors to CAR-T cell therapies, the narrative unravels the potential of harnessing the body's immune system to combat and eradicate diseases.",
                  "Precision medicine stands at the forefront of healthcare innovation, revolutionizing medical treatment by customizing interventions based on individual patient characteristics. Leveraging genetic, genomic, and patient-specific data, healthcare practitioners tailor treatments to maximize efficacy while minimizing side effects. Genomic sequencing plays a pivotal role, identifying genetic variations influencing disease susceptibility, progression, and treatment response. This approach extends beyond oncology, impacting cardiovascular diseases, neurodegenerative disorders, and rare genetic conditions. Precision medicine's promise lies in effective, targeted therapies aligned with each patient's unique genetic makeup. As technology advances, precision medicine holds the potential to reshape healthcare, providing personalized, patient-centric diagnostics and therapeutics for improved outcomes.",
                  "Artificial intelligence (AI) is revolutionizing the field of medicine, offering innovative solutions for diagnostics, treatment planning, and patient care. This text explores how machine learning algorithms and AI-driven technologies are being integrated into medical practices. From medical imaging interpretation to drug discovery, the impact of AI on various facets of medicine is discussed, emphasizing the potential for more accurate diagnostics and personalized treatment approaches."]
sports_texts = ["E-Sports, once considered a niche pastime, has rapidly evolved into a global phenomenon that captivates millions of enthusiasts worldwide. This text explores the journey of E-Sports, from its humble beginnings to mainstream recognition. Examining the rise of professional gaming leagues, the impact of streaming platforms, and the competitive spirit fueling this digital revolution, we delve into how E-Sports has redefined the landscape of competitive entertainment.", 
                "Technological advancements are reshaping the world of sports, enhancing both athlete performance and fan experiences. From wearable devices that track player metrics to virtual reality applications offering immersive fan engagement, this text explores the intersection of sports and technology. As athletes leverage data-driven insights for training optimization, and fans embrace interactive digital experiences, the sports industry undergoes a dynamic transformation that transcends traditional boundaries.",
                "Women's participation and influence in sports have undergone a remarkable evolution, challenging stereotypes and breaking barriers. This text celebrates the achievements of women in sports, examining the strides made in diverse disciplines. From iconic athletes who have shattered records to the growing popularity of women's sports leagues, we explore how these achievements contribute to a more inclusive and equitable sporting landscape.",
                "Extreme sports defy conventional notions of athletic pursuits, pushing individuals to their physical and mental limits. This text delves into the world of extreme sports, exploring the adrenaline-fueled experiences of participants. From base jumping and snowboarding to skateboarding and parkour, we examine how these daring activities have captured the imagination of thrill-seekers worldwide and become cultural phenomena in their own right.",
                "Major sporting events, such as the Olympics and World Cup, transcend athletic competitions to become global spectacles with far-reaching impacts. This text analyzes the influence of these mega-events on host cities, economies, and international relations. From the infrastructure developments spurred by hosting to the cultural exchanges facilitated by diverse participation, we explore the multifaceted dimensions of major sporting events and their enduring legacies."]

### Function to generate summaries for a list of texts

In [4]:
def generate_summaries(texts, model, tokenizer):
    summaries = []
    for text in texts:
        # Tokenize the input text
        input_ids = tokenizer.encode(text, return_tensors="pt")

        # Generate the summary
        summary_ids = model.generate(input_ids, max_length=150, num_beams=4, length_penalty=2.0)

        # Decode and append the generated summary
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary_text)

    return summaries

### Function for evaluating metrics

In [7]:
def metrics(sum1,sum2,sum3,model):
    #Create a dictionary to store DataFrames for each domain
    domain_dataframes_m2 = {}

    # Evaluate and append metrics for each domain
    for domain, references, summaries in zip(domains, [politics_texts, medicine_texts, sports_texts], [sum1, sum2, sum3]):
        # Create a DataFrame for the current domain
        data = []

        for i, (reference, summary) in enumerate(zip(references, summaries), 1):
            # BLEU Score
            bleu = corpus_bleu([[ref.split()] for ref in [reference]], [cand.split() for cand in [summary]])

            # BERTScore
            _, _, bert_score_value = score([reference], [summary], lang="en", model_type="bert-base-uncased", verbose=False)

            # METEOR Score
            tokenized_reference = word_tokenize(reference)
            tokenized_summary = word_tokenize(summary)
            meteor = meteor_score([tokenized_reference], tokenized_summary)

            # Jaccard Similarity
            jaccard_sim = 1 - jaccard_distance(set(reference.split()), set(summary.split()))

            # Flesch Reading Ease Score
            reading_ease = flesch_reading_ease(summary)

            # Append metrics to the data list
            data.append([domain,f"Text {i}", model, bleu, bert_score_value.item(), meteor, jaccard_sim, reading_ease])

        # Create the DataFrame for the current domain
        columns = ["Domain","Text", "Model","BLEU Score", "BERTScore", "METEOR Score", "Jaccard Similarity", "Flesch Reading Ease"]
        domain_df_m2 = pd.DataFrame(data, columns=columns)

        # Store the domain DataFrame in the dictionary
        domain_dataframes_m2[domain] = domain_df_m2

    # Concatenate DataFrames for all domains
    all_domains_df_m2 = pd.concat(domain_dataframes_m2.values(), ignore_index=True)
    
    return all_domains_df_m2

## Model 1
```facebook/bart-large-cnn```: BART model pre-trained on English language, and fine-tuned on CNN Daily Mail.
- Link: https://huggingface.co/facebook/bart-large-cnn 

In [2]:
bart_model_name = "facebook/bart-large-cnn"
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)
bart_tokenizer = BartTokenizer.from_pretrained(bart_model_name)

In [5]:
# Generate summaries for each domain
politics_summaries = generate_summaries(politics_texts, bart_model, bart_tokenizer)
medicine_summaries = generate_summaries(medicine_texts,bart_model, bart_tokenizer)
sports_summaries = generate_summaries(sports_texts,bart_model, bart_tokenizer)

In [6]:
domains = ["Politics", "Medicine", "Sports"]
mod1='M1'

In [8]:
df1=metrics(politics_summaries,medicine_summaries,sports_summaries,mod1)

In [9]:
df1

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease
0,Politics,Text 1,M1,0.211729,0.789640,0.423304,0.402299,51.34
1,Politics,Text 2,M1,0.577965,0.910529,0.680344,0.721311,27.83
2,Politics,Text 3,M1,0.512393,0.909076,0.647649,0.650794,3.76
3,Politics,Text 4,M1,0.452470,0.861453,0.603071,0.600000,11.92
4,Politics,Text 5,M1,0.597071,0.888432,0.709254,0.701493,0.42
5,Medicine,Text 1,M1,0.860911,0.964507,0.909076,0.892857,-6.74
6,Medicine,Text 2,M1,0.693563,0.916776,0.778031,0.706897,29.55
7,Medicine,Text 3,M1,0.913862,0.985776,0.955362,0.934783,28.54
8,Medicine,Text 4,M1,0.224553,0.812943,0.398042,0.423913,-18.88
9,Medicine,Text 5,M1,0.796789,0.956066,0.839393,0.836364,19.67


## Model 2
```philschmid/bart-large-cnn-samsum```: This model was trained using Amazon SageMaker and the new Hugging Face Deep Learning container.
- link: https://huggingface.co/philschmid/bart-large-cnn-samsum

In [10]:
model_name_samsum = "philschmid/bart-large-cnn-samsum"
model_samsum = BartForConditionalGeneration.from_pretrained(model_name_samsum)
tokenizer_samsum = BartTokenizer.from_pretrained(model_name_samsum)

In [11]:
politics_summaries_samsum = generate_summaries(politics_texts, model_samsum, tokenizer_samsum)
medicine_summaries_samsum = generate_summaries(medicine_texts,model_samsum, tokenizer_samsum)
sports_summaries_samsum = generate_summaries(sports_texts,model_samsum, tokenizer_samsum)

In [12]:
mod2='M2'

In [13]:
df2=metrics(politics_summaries_samsum,medicine_summaries_samsum,sports_summaries_samsum,mod2)

In [14]:
df2

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease
0,Politics,Text 1,M2,0.227377,0.803003,0.430113,0.435294,39.33
1,Politics,Text 2,M2,0.497020,0.857119,0.577496,0.569231,36.28
2,Politics,Text 3,M2,0.352068,0.861470,0.460591,0.470588,30.57
3,Politics,Text 4,M2,0.436246,0.877809,0.517539,0.588235,11.62
4,Politics,Text 5,M2,0.320115,0.838422,0.546458,0.478873,-3.99
5,Medicine,Text 1,M2,0.324496,0.766040,0.523897,0.393939,-10.08
6,Medicine,Text 2,M2,0.542213,0.888175,0.772670,0.580645,29.14
7,Medicine,Text 3,M2,0.692628,0.921811,0.809320,0.720000,30.16
8,Medicine,Text 4,M2,0.217132,0.820139,0.428191,0.414894,-3.28
9,Medicine,Text 5,M2,0.642195,0.924053,0.737276,0.714286,4.78


## Model 3
```sshleifer/distilbart-cnn-12-6```: A distilled version of the BART  model, specifically fine-tuned for the CNN/DailyMail summarization task.
- link: https://huggingface.co/sshleifer/distilbart-cnn-12-6

In [15]:
model_name_distilbart = "sshleifer/distilbart-cnn-12-6"
model_distilbart = BartForConditionalGeneration.from_pretrained(model_name_distilbart)
tokenizer_distilbart = BartTokenizer.from_pretrained(model_name_distilbart)

In [16]:
politics_summaries_distilbart = generate_summaries(politics_texts, model_distilbart, tokenizer_distilbart)
medicine_summaries_distilbart = generate_summaries(medicine_texts,model_distilbart, tokenizer_distilbart)
sports_summaries_distilbart = generate_summaries(sports_texts,model_distilbart, tokenizer_distilbart)

In [17]:
mod3='M3'

In [18]:
df3=metrics(politics_summaries_distilbart,medicine_summaries_distilbart,sports_summaries_distilbart,mod3)

In [19]:
df3

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease
0,Politics,Text 1,M3,0.510490,0.873370,0.632050,0.630952,7.56
1,Politics,Text 2,M3,0.690484,0.921841,0.774255,0.770492,25.49
2,Politics,Text 3,M3,0.489013,0.897466,0.622908,0.629032,-3.15
3,Politics,Text 4,M3,0.652749,0.917092,0.716602,0.761905,8.88
4,Politics,Text 5,M3,0.315533,0.779176,0.499958,0.471429,-11.74
5,Medicine,Text 1,M3,0.817444,0.968421,0.877237,0.890909,-6.33
6,Medicine,Text 2,M3,0.582014,0.885534,0.665888,0.678571,16.32
7,Medicine,Text 3,M3,0.722751,0.903341,0.771990,0.686275,37.30
8,Medicine,Text 4,M3,0.299121,0.841600,0.494998,0.494505,-3.99
9,Medicine,Text 5,M3,0.743663,0.933119,0.784681,0.781818,28.84


## Model 4
```google/pegasus-large```: The "google/pegasus-large" variant refers to a larger version of the Pegasus model, designed for more complex and extensive text summarization tasks. Leveraging pre-training on large-scale datasets, Pegasus-large is adept at distilling long-form content into concise and coherent summaries, making it a powerful tool for various natural language processing applications, including document summarization and content abstraction.
- link: https://huggingface.co/google/pegasus-large

In [20]:
# Load the pre-trained Pegasus model and tokenizer
model_name = "google/pegasus-large"
pegasus_large_model = PegasusForConditionalGeneration.from_pretrained(model_name)
pegasus_large_tokenizer = PegasusTokenizer.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
politics_summaries_pegasus_large = generate_summaries(politics_texts, pegasus_large_model, pegasus_large_tokenizer)
medicine_summaries_pegasus_large = generate_summaries(medicine_texts,pegasus_large_model, pegasus_large_tokenizer)
sports_summaries_pegasus_large = generate_summaries(sports_texts,pegasus_large_model, pegasus_large_tokenizer)

In [22]:
mod4='M4'

In [23]:
df4=metrics(politics_summaries_pegasus_large,medicine_summaries_pegasus_large,sports_summaries_pegasus_large,mod4)

In [24]:
df4

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease
0,Politics,Text 1,M4,0.027087,0.731026,0.229983,0.265060,31.21
1,Politics,Text 2,M4,0.028566,0.749594,0.230932,0.295082,2.44
2,Politics,Text 3,M4,0.042515,0.752956,0.227887,0.306452,1.43
3,Politics,Text 4,M4,0.049787,0.773049,0.237534,0.317460,-0.61
4,Politics,Text 5,M4,0.117319,0.780843,0.335564,0.393939,0.76
5,Medicine,Text 1,M4,0.086774,0.781692,0.265442,0.327273,-6.02
6,Medicine,Text 2,M4,0.162321,0.804736,0.361978,0.363636,15.31
7,Medicine,Text 3,M4,0.113441,0.764729,0.313533,0.369565,11.92
8,Medicine,Text 4,M4,0.016573,0.701152,0.173378,0.219780,-24.97
9,Medicine,Text 5,M4,0.299124,0.831433,0.458446,0.500000,8.20


## Model 5
```google/pegasus-cnn_dailymail```: This model, based on Pegasus architecture, is pre-trained on a large dataset from the CNN/DailyMail news summarization task. It is designed for text summarization, enabling the generation of concise and coherent summaries for news articles. The model excels in capturing the essence of lengthy texts and producing informative and coherent abstractive summaries.
- link: https://huggingface.co/google/pegasus-cnn_dailymail

In [25]:
# Load the pre-trained Pegasus model and tokenizer
model_name = "google/pegasus-cnn_dailymail"
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
politics_summaries_pegasus = generate_summaries(politics_texts, pegasus_model, pegasus_tokenizer)
medicine_summaries_pegasus = generate_summaries(medicine_texts,pegasus_model, pegasus_tokenizer)
sports_summaries_pegasus = generate_summaries(sports_texts,pegasus_model, pegasus_tokenizer)

In [27]:
mod5='M5'

In [28]:
df5=metrics(politics_summaries_pegasus,medicine_summaries_pegasus,sports_summaries_pegasus,mod5)

In [29]:
df5

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease
0,Politics,Text 1,M5,0.389898,0.836126,0.555907,0.500000,-29.70
1,Politics,Text 2,M5,0.714054,0.921090,0.828213,0.712121,-33.42
2,Politics,Text 3,M5,0.466458,0.870830,0.542996,0.609375,-19.55
3,Politics,Text 4,M5,0.871130,0.949758,0.930920,0.863636,-61.51
4,Politics,Text 5,M5,0.449755,0.874810,0.522667,0.602941,-48.99
5,Medicine,Text 1,M5,0.740284,0.936934,0.846353,0.793103,-51.02
6,Medicine,Text 2,M5,0.686112,0.902689,0.748180,0.732143,-10.08
7,Medicine,Text 3,M5,0.625950,0.847547,0.840668,0.857143,-29.02
8,Medicine,Text 4,M5,0.316154,0.809469,0.497116,0.494624,-38.50
9,Medicine,Text 5,M5,0.679196,0.915932,0.773545,0.736842,-22.60


## Concatinating and Arranging Dataframes

In [30]:
result = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [31]:
result

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease
0,Politics,Text 1,M1,0.211729,0.789640,0.423304,0.402299,51.34
1,Politics,Text 2,M1,0.577965,0.910529,0.680344,0.721311,27.83
2,Politics,Text 3,M1,0.512393,0.909076,0.647649,0.650794,3.76
3,Politics,Text 4,M1,0.452470,0.861453,0.603071,0.600000,11.92
4,Politics,Text 5,M1,0.597071,0.888432,0.709254,0.701493,0.42
...,...,...,...,...,...,...,...,...
70,Sports,Text 1,M5,0.888568,0.961433,0.940823,0.888889,-26.32
71,Sports,Text 2,M5,0.886761,0.958927,0.938333,0.896552,-42.22
72,Sports,Text 3,M5,0.749363,0.930064,0.842557,0.788462,-0.26
73,Sports,Text 4,M5,0.720347,0.916921,0.821537,0.740741,-14.14


Group by "Domain" and then by "Text"

In [32]:
grouped_df = result.groupby(['Domain', 'Text'])
group_dfs = []
for (domain, text), group in grouped_df:
    group_dfs.append(group)
    
concatenated_df = pd.concat(group_dfs, ignore_index=True)

In [33]:
Politics=concatenated_df[concatenated_df['Domain']=='Politics']
Medicine=concatenated_df[concatenated_df['Domain']=='Medicine']
Sports=concatenated_df[concatenated_df['Domain']=='Sports']

## Applying Topsis
We extract topsis() function from our previous assignment and made some changes according to this problem.

In [35]:
def topsis(data,weights,impacts):
    
    #Step 1
    #removing first 3 columns that's usually the name of models/criteria, Text and Domain
    df=data.iloc[:, 3:]
    
    #Step 2
    #Calculate root of square of sum for each column
    rss=[]
    for j in range(0,df.shape[1]):
        l=[]
        for i in range(0,df.shape[0]):
            l.append(df.iloc[i,j]**2)
        rss.append((sum(l))**0.5)
    
    #Step 3
    #Calculate normalized performance values
    for i in range(0,df.shape[1]):
        for j in range(0,df.shape[0]):
            df.iloc[j,i]=df.iloc[j,i]/rss[i]
    
    #Step 4
    #Calculate weighted normalised decision matrix
    for i in range(0,df.shape[1]):
        for j in range(0,df.shape[0]):
            df.iloc[j,i]=df.iloc[j,i]/weights[i]
    
    #Step 5
    #Extract ideal best and ideal worst for each column according to impact
    ideal_best=[]
    ideal_worst=[]
    for i in range(0,df.shape[1]):
            if impacts[i] == '+':
                ideal_best.append(max(df.iloc[:,i]))
                ideal_worst.append(min(df.iloc[:,i]))
            elif impacts[i] == '-':
                ideal_best.append(min(df.iloc[:,i]))
                ideal_worst.append(max(df.iloc[:,i]))
    
    #Step 6
    #Calculate euclidean distance for both ideal best and ideal worst value
    Sp=[]
    Sn=[]
    for i in range(0,df.shape[0]):
        l1=[]
        l2=[]
        for j in range(0,df.shape[1]):
            l1.append((df.iloc[i,j]-ideal_best[j])**2)
            l2.append((df.iloc[i,j]-ideal_worst[j])**2)
        Sp.append(sum(l1)**0.5)
        Sn.append(sum(l2)**0.5)
    
    #Step 7
    #Calculate performance score
    p=[]
    for i in range(0,len(Sp)):
        p.append(Sn[i]/(Sn[i]+Sp[i]))
    
    #Step 8
    #Create DataFrame
    df_new=pd.DataFrame(data)
    df_new['Performance Score']=p
    
    #Step 9
    #Calculate Rank based on performance score and add it to the dataframe
    df_new['Rank'] = df_new['Performance Score'].rank(ascending=False)
    
    return df_new

Specify impacts and weights for each parameters. We will add equal weights for each parameter and for impacts:
#### BLEU Score:
Impact: Higher BLEU scores indicate better performance in terms of n-gram overlap between the generated summary and reference summaries.

#### BERTScore:
Impact: Higher BERTScore values indicate better semantic similarity between the generated summary and reference summaries.

#### METEOR Score:
Impact: Higher METEOR scores indicate better overall linguistic quality and fluency of the generated summary.

#### Jaccard Similarity:
Impact: Higher Jaccard similarity values indicate better set similarity between the words in the generated summary and reference summaries.

#### Flesch Reading Ease:
Impact: Higher Flesch Reading Ease scores indicate easier-to-read text, with lower scores representing more complex text.

```So the impacts for each parameter will be '+'```

In [36]:
weights=[1,1,1,1,1]
impacts=['+','+','+','+','+']

In [37]:
texts = ['Text 1', 'Text 2', 'Text 3', 'Text 4', 'Text 5']
all_results = []

for text in texts:
    data = Politics[Politics['Text'] == text]
    result = topsis(data, weights, impacts)
    all_results.append(result)

# Concatenate all DataFrames
final_result = pd.concat(all_results, ignore_index=True)

In [38]:
final_result

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease,Performance Score,Rank
0,Politics,Text 1,M1,0.211729,0.789640,0.423304,0.402299,51.34,0.680034,1.0
1,Politics,Text 1,M2,0.227377,0.803003,0.430113,0.435294,39.33,0.655940,2.0
2,Politics,Text 1,M3,0.510490,0.873370,0.632050,0.630952,7.56,0.636136,3.0
3,Politics,Text 1,M4,0.027087,0.731026,0.229983,0.265060,31.21,0.465911,4.0
4,Politics,Text 1,M5,0.389898,0.836126,0.555907,0.500000,-29.70,0.375468,5.0
5,Politics,Text 2,M1,0.577965,0.910529,0.680344,0.721311,27.83,0.850638,2.0
6,Politics,Text 2,M2,0.497020,0.857119,0.577496,0.569231,36.28,0.811589,3.0
7,Politics,Text 2,M3,0.690484,0.921841,0.774255,0.770492,25.49,0.870325,1.0
8,Politics,Text 2,M4,0.028566,0.749594,0.230932,0.295082,2.44,0.380537,5.0
9,Politics,Text 2,M5,0.714054,0.921090,0.828213,0.712121,-33.42,0.400066,4.0


In [39]:
final_result[final_result['Rank']==1.0]

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease,Performance Score,Rank
0,Politics,Text 1,M1,0.211729,0.789640,0.423304,0.402299,51.34,0.680034,1.0
7,Politics,Text 2,M3,0.690484,0.921841,0.774255,0.770492,25.49,0.870325,1.0
11,Politics,Text 3,M2,0.352068,0.861470,0.460591,0.470588,30.57,0.836425,1.0
17,Politics,Text 4,M3,0.652749,0.917092,0.716602,0.761905,8.88,0.839585,1.0
20,Politics,Text 5,M1,0.597071,0.888432,0.709254,0.701493,0.42,0.994383,1.0


Analysis of best model for each text:
- Text 1: Model M1
- Text 2: Model M3
- Text 3: Model M2
- Text 4: Model M3
- Text 5: Model M1

Do similar for Medicine and Sports Domain

In [40]:
all_results2 = []
for text in texts:
    data = Medicine[Medicine['Text'] == text]
    result2 = topsis(data, weights, impacts)
    all_results2.append(result2)

# Concatenate all DataFrames
final_result2 = pd.concat(all_results2, ignore_index=True)

In [41]:
final_result2

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease,Performance Score,Rank
0,Medicine,Text 1,M1,0.860911,0.964507,0.909076,0.892857,-6.74,0.988039,1.0
1,Medicine,Text 1,M2,0.324496,0.766040,0.523897,0.393939,-10.08,0.590479,3.0
2,Medicine,Text 1,M3,0.817444,0.968421,0.877237,0.890909,-6.33,0.968338,2.0
3,Medicine,Text 1,M4,0.086774,0.781692,0.265442,0.327273,-6.02,0.525736,4.0
4,Medicine,Text 1,M5,0.740284,0.936934,0.846353,0.793103,-51.02,0.433816,5.0
5,Medicine,Text 2,M1,0.693563,0.916776,0.778031,0.706897,29.55,0.982206,1.0
6,Medicine,Text 2,M2,0.542213,0.888175,0.772670,0.580645,29.14,0.850613,2.0
7,Medicine,Text 2,M3,0.582014,0.885534,0.665888,0.678571,16.32,0.702160,3.0
8,Medicine,Text 2,M4,0.162321,0.804736,0.361978,0.363636,15.31,0.451629,4.0
9,Medicine,Text 2,M5,0.686112,0.902689,0.748180,0.732143,-10.08,0.402078,5.0


In [42]:
final_result2[final_result2['Rank']==1.0]

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease,Performance Score,Rank
0,Medicine,Text 1,M1,0.860911,0.964507,0.909076,0.892857,-6.74,0.988039,1.0
5,Medicine,Text 2,M1,0.693563,0.916776,0.778031,0.706897,29.55,0.982206,1.0
10,Medicine,Text 3,M1,0.913862,0.985776,0.955362,0.934783,28.54,0.895067,1.0
17,Medicine,Text 4,M3,0.299121,0.841600,0.494998,0.494505,-3.99,0.965733,1.0
22,Medicine,Text 5,M3,0.743663,0.933119,0.784681,0.781818,28.84,0.954393,1.0


Analysis of best model for each text:

- Text 1: Model M1
- Text 2: Model M1
- Text 3: Model M1
- Text 4: Model M3
- Text 5: Model M3

In [43]:
all_results3 = []
for text in texts:
    data = Sports[Sports['Text'] == text]
    result3 = topsis(data, weights, impacts)
    all_results3.append(result3)

# Concatenate all DataFrames
final_result3 = pd.concat(all_results3, ignore_index=True)

In [44]:
final_result3

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease,Performance Score,Rank
0,Sports,Text 1,M1,0.550703,0.902387,0.719784,0.600000,30.87,0.756470,2.0
1,Sports,Text 1,M2,0.570002,0.889327,0.673543,0.690909,30.16,0.768856,1.0
2,Sports,Text 1,M3,0.623774,0.907483,0.723912,0.698113,22.75,0.752912,3.0
3,Sports,Text 1,M4,0.356561,0.823281,0.512815,0.500000,5.16,0.444937,4.0
4,Sports,Text 1,M5,0.888568,0.961433,0.940823,0.888889,-26.32,0.341705,5.0
5,Sports,Text 2,M1,0.608628,0.880662,0.734178,0.789474,4.78,0.768643,2.0
6,Sports,Text 2,M2,0.489662,0.905319,0.639184,0.649123,0.42,0.666682,3.0
7,Sports,Text 2,M3,0.553786,0.874625,0.757400,0.661290,16.18,0.808548,1.0
8,Sports,Text 2,M4,0.038133,0.742690,0.224043,0.267857,13.95,0.567127,4.0
9,Sports,Text 2,M5,0.886761,0.958927,0.938333,0.896552,-42.22,0.423011,5.0


In [45]:
final_result3[final_result3['Rank']==1.0]

,Domain,Text,Model,BLEU Score,BERTScore,METEOR Score,Jaccard Similarity,Flesch Reading Ease,Performance Score,Rank
1,Sports,Text 1,M2,0.570002,0.889327,0.673543,0.690909,30.16,0.768856,1.0
7,Sports,Text 2,M3,0.553786,0.874625,0.757400,0.661290,16.18,0.808548,1.0
12,Sports,Text 3,M3,0.931500,0.947284,0.870664,1.000000,34.26,0.809772,1.0
17,Sports,Text 4,M3,0.826486,0.951950,0.873145,0.811321,28.13,0.858843,1.0
20,Sports,Text 5,M1,0.764503,0.935127,0.831871,0.814815,20.08,0.784647,1.0


Analysis of best model for each text:

- Text 1: Model M2
- Text 2: Model M3
- Text 3: Model M3
- Text 4: Model M3
- Text 5: Model M1

From Our Analysis of each Domain we can conclude the following:

| Domain      | Best Model |
|-------------|------------|
| Politics    | M1,M3      |
| Medicine    | M1         |
| Sports      | M3         |

- M1: ```facebook/bart-large-cnn```
- M3: ```sshleifer/distilbart-cnn-12-6```